In [1]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import binary_opening
import netCDF4 as nc

%matplotlib inline

In [23]:
def clean_layer(layer):
    structure = [[0, 1, 0], [1, 1, 1], [0, 1, 0]]
    return binary_opening(layer, structure=structure)

def get_cleaned_gpi(gpi):
    # Apply binary opening to remove speckle noise
    gpis_cleaned = []
    for i in range(len(gpi)):
        gpi20 = gpi[i].astype(int)
        gpi_cleaned = clean_layer(gpi20)
        gpis_cleaned.append(gpi_cleaned)
    gpis_cleaned = np.array(gpis_cleaned).astype(bool)
    return gpis_cleaned

In [ ]:
idir = f'/data1/{os.getenv("USER")}/foccus/greenland/'
drift_files = sorted(glob.glob(f'{idir}/drift_*.npz'))
print(len(drift_files), drift_files[-1])

In [25]:
landmask = np.load(f'{idir}/landmask.npz')['landmask'][::10, ::10]
landmask = np.ma.masked_array(landmask, landmask == 0)

In [26]:
drift_cols = []
drift_rows = []
drift_cors = []
for i, drift_file in enumerate(drift_files):
    with np.load(drift_file) as drift:
        drift_col = drift['drift_col']
        drift_row = drift['drift_row']
        drift_cor = drift['drift_cor']
    drift_cols.append(drift_col)
    drift_rows.append(drift_row)
    drift_cors.append(drift_cor)

drift_cols = np.array(drift_cols)
drift_rows = np.array(drift_rows)
drift_cors = np.array(drift_cors)
drift_spds = np.hypot(drift_cols, drift_rows)

In [ ]:
gpi = np.isfinite(drift_cors*drift_spds)
plt.hist2d(drift_cors[gpi], drift_spds[gpi], 100, [[0, 1], [0, 0.2]], cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
min_cor = 0.2
max_drift = 0.01
fast_ices = (drift_cors > min_cor) & (drift_spds < max_drift)
fast_ices_clean = get_cleaned_gpi(fast_ices)
fast_ices_sum = np.nansum(fast_ices_clean, axis=0)
sum_obs = np.nansum(np.isfinite(drift_cors), axis=0)
imsh = plt.imshow(fast_ices_sum / sum_obs, vmin=0, vmax=0.5, cmap='jet', interpolation='nearest')
plt.imshow(landmask, cmap='gray', alpha=0.2, interpolation='nearest')
plt.show()

In [ ]:
step = len(drift_files) // 5

spd_lim = [0.0, 0.2]
cor_lim = [0.0, 1.0]
alpha = 0.5
plt_stp = 10

show_drift_files = drift_files[1::step]
for drift_file in show_drift_files:
    print(drift_file)
    mosaic_file = drift_file.replace('drift', 'mosaic')
    with np.load(drift_file) as drift:
        drift_col = drift['drift_col']
        drift_row = drift['drift_row']
        drift_cor = drift['drift_cor']
    sar_primary = np.load(mosaic_file)['sar_primary']
    drift_spd = np.hypot(drift_col, drift_row)

    fast_ice = (drift_cor > min_cor) & (drift_spd < max_drift)
    fast_ice_clean = clean_layer(fast_ice)
    fast_ice_clean_masked = np.ma.masked_array(fast_ice_clean, fast_ice_clean == 0)
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    for ax in axs:
        ax.imshow(sar_primary[::plt_stp, ::plt_stp], cmap='gray', interpolation='none')

    axs[0].imshow(drift_spd, cmap='jet', alpha=alpha, clim=spd_lim, interpolation='none', extent=[0, sar_primary.shape[1]/plt_stp, sar_primary.shape[0]/plt_stp, 0])
    axs[1].imshow(drift_cor, cmap='jet', alpha=alpha, clim=cor_lim, interpolation='none', extent=[0, sar_primary.shape[1]/plt_stp, sar_primary.shape[0]/plt_stp, 0])
    axs[2].imshow(fast_ice_clean_masked, cmap='coolwarm', alpha=alpha, clim=[0,1], interpolation='none', extent=[0, sar_primary.shape[1]/plt_stp, sar_primary.shape[0]/plt_stp, 0])
    plt.show()


In [9]:
fast_ices_export = np.zeros(fast_ices_clean.shape, dtype=np.uint8)
fast_ices_export[:, landmask==1] = 1          # Land mask
fast_ices_export[np.isfinite(drift_cors)] = 2 # Not fast ice
fast_ices_export[fast_ices_clean] = 3         # Fast ice


In [17]:
fast_ice_metadata = dict(
    long_name = "Detection of fast ice - floating sea ice with zero motion",
    standard_name = "sea_ice_classification",
    units = "1",
    grid_mapping = "Polar_Stereographic_Grid",
    coordinates = "y x",
    descriptions = "0 - no data, 1 - land, 2 - not fast ice, 3 - fast ice",
)

y_metadata = dict(
    axis = "y",
    units = "m",
    long_name = "y coordinate in Cartesian system",
    standard_name = "projection_y_coordinate",
)

Polar_Stereographic_Grid_metadata = dict(
    grid_mapping_name = "polar_stereographic" ,
    false_easting = 0. ,
    false_northing = 0. ,
    area_extent = "\'(-200000 -1500000 200000 -1000000)\'" ,
    semi_major_axis = 6378273. ,
    semi_minor_axis = 6356889.44891 ,
    straight_vertical_longitude_from_pole = 18. ,
    latitude_of_projection_origin = 90. ,
    standard_parallel = 70. ,
    proj4_string = "+proj=stere +lon_0=18 +lat_0=90 +lat_ts=70 +ellps=WGS84 +datum=WGS84 +units=m" ,
)

x_metadata = dict(
    axis = "x",
    units = "m",
    long_name = "x coordinate in Cartesian system",
    standard_name = "projection_x_coordinate",
)


global_metadata = dict(
    title = "Level-2 Fast Sea Ice Detection" ,
    product_id = "FOCCUS-NERSC-Fast-Ice" ,
    instrument_type = "C-band SAR" ,
    platform_name = "Sentinel-1" ,
    easternmost_longitude = -42. ,
    westernmost_longitude = 40. ,
    northernmost_latitude = 80. ,
    southernmost_latitude = 57. ,
    product_name = "foccus_nersc_fast_ice" ,
    product_status = "archive" ,
    abstract = "Fast sea ice is detected on pairs of Setinel-1 SAR images using the NERSC algorithm. The product is a mask with 1 for fast ice and 0 for no fast ice and 2 for land. The product is generated by the Nansen Environmental and Remote Sensing Center (NERSC) and is distributed in the FOCCUS Zenodo archive." ,
    topiccategory = "Oceans Climatology Meteorology Atmosphere" ,
    keywords = "Fast Ice, Sea Ice Concentration, Sea Ice, Oceanography, Meteorology, Climate, Remote Sensing" ,
    gcmd_keywords = [
        "Cryosphere > Sea Ice > Sea Ice Concentration\n",
        "Oceans > Sea Ice > Sea Ice Concentration\n",
        "Geographic Region > Northern Hemisphere\n",
        "Vertical Location > Sea Surface\n"
    ],
    activity_type = "Space borne instrument" ,
    start_date = "START_DATE" ,
    stop_date = "STOP_DATE" ,
    project_name = "EU FOCCUS" ,
    distribution_statement = "Free" ,
    copyright_statement = "Copyright 2024 NERSC" ,
    Conventions = "CF-1.6" ,
    format_version = "v0.1",
    summary = "Fast ice extent is derived from sequences of SAR images using sea ice drift estimates",
    source_platform_category_code = "satellite",
    platform = "Sentinel-1",
    platform_type = "polar Earth orbit satellite",
    platform_vocabulary = "GCMD",
    instrument = "C-SAR",
    instrument_vocabulary = "GCMD",
    institution = "Nansen Environmental and Remote Sensing Center",
    institution_edmo_code = "1413",
    references = "Korosov, A.A.; Rampal, P. A Combination of Feature Tracking and Pattern Matching with Optimal Parameterization for Sea Ice Drift Retrieval from SAR Data. Remote Sens. 2017, 9, 258. https://doi.org/10.3390/rs9030258",
    product_version = "1.0",
    comment = "First version of the fast ice extent product",
    cdm_data_type = "grid",
    area = "Greenland coast and Svalbard",
    geospatial_lat_min = 57,
    geospatial_lat_max = 80,
    geospatial_lon_min = -42,
    geospatial_lon_max = 40,
    geospatial_vertical_min = 0,
    geospatial_vertical_max = 0,
    geospatial_vertical_resolution = 0,
    update_interval = "irregular",
    processing_level = "L2",
    creator_url = "https://nersc.no",
    contributor_name = "0000-0002-3601-1161",
    contributor_url = "https://nersc.no",
    contributor_email = "post@nersc.no",
    method = "sea ice drift",
    track_id =  "NASA Global Change Master Directory (GCMD) Science Keywords",
)


In [ ]:
from datetime import datetime

for i in range(len(drift_files)):
    output_filename = drift_files[i].replace('drift_', 'foccus_nersc_fast_ice_').replace('npz', 'nc')
    name = os.path.basename(drift_files[i]).split('.')[0]
    start_date = datetime.strptime(name.split('_')[1], '%Y%m%dT%H%M%S')
    stop_date = datetime.strptime(name.split('_')[2], '%Y%m%dT%H%M%S')
    global_metadata['start_date'] = start_date.strftime('%Y-%m-%d %H:%M:%S')
    global_metadata['stop_date'] = stop_date.strftime('%Y-%m-%d %H:%M:%S')
    print(output_filename)
    with nc.Dataset(output_filename, 'w', format='NETCDF4') as dst:
        # Create x and y dimensions with specified ranges
        x = dst.createDimension('x', fast_ices_export.shape[1])
        y = dst.createDimension('y', fast_ices_export.shape[2])

        x_var = dst.createVariable('x', 'f4', ('x',))
        y_var = dst.createVariable('y', 'f4', ('y',))
        x_var[:] = np.linspace(-200000, 200000, fast_ices_export.shape[1])
        y_var[:] = np.linspace(-1500000, -1000000, fast_ices_export.shape[2])
        for items in y_metadata.items():
            y_var.setncattr(*items)
        for items in x_metadata.items():
            x_var.setncattr(*items)

        # Create CRS variable
        crs_var = dst.createVariable('Polar_Stereographic_Grid', 'c')
        for items in Polar_Stereographic_Grid_metadata.items():
            crs_var.setncattr(*items)

        # Create variable
        fast_ice_var = dst.createVariable('fast_ice', 'u1', ('x', 'y'), zlib=True, complevel=4)

        # Assign data to variable
        fast_ice_var[:, :] = fast_ices_export[i]
        for items in fast_ice_metadata.items():
            fast_ice_var.setncattr(*items)

        for items in global_metadata.items():
            dst.setncattr(*items)
    break

In [ ]:
!ncdump -h {output_filename}
